# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract text from a technical PDF
2. Prepare training data for causal language modelling (CLM)
3. Fine-tune a language (llama 3.2 3b) base model using QLoRA
4. Apply the QLoRA and the learned weights to the instruct model
5. Answer some test questions

In [1]:
# Clone the git repo to access the utilities
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

Cloning into 'mistralpeft'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 110 (delta 67), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 7.65 MiB | 17.37 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [2]:
# Make sure to be on the repo directory and pull
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git pull 

/kaggle/working/mistralpeft
Already up to date.


In [3]:
# Install the required packages from pypi
!pip install -r requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.6 MB/s eta 0:00:00


In [4]:
# Load packages
from transformers import Trainer, TrainingArguments, AutoTokenizer
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftConfig, PeftModel
from datasets import Dataset

from mistralpeft.utils import TextExtractor, CLMPreprocessor, generate_response

In [5]:
# Set environment
# Switch of tracking with Weights And Biases (it requires an account)
os.environ["WANDB_DISABLED"] = "true"

In [6]:
# Login to HuggingFace using Kaggle's secrets to be able to download models
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
login(secret_value_0) 

## 1. Extract Sentences from PDF
Several PDFs from my former research group at university (Thermo-Fluiddynamics Group, Prof. Polifke) are chosen to form the corpus

In [7]:
# TextExtractor is a simple ETL class to acquire a text corpus
pdf_files = [
    "Dissertation.pdf",
]
    
pdf_urls = [
    # "https://mediatum.ub.tum.de/doc/1360567/1360567.pdf",
    # "https://mediatum.ub.tum.de/doc/1601190/1601190.pdf",
    # "https://mediatum.ub.tum.de/doc/1597610/1597610.pdf"
    # "https://mediatum.ub.tum.de/doc/1584750/1584750.pdf",
    # "https://mediatum.ub.tum.de/doc/1484812/1484812.pdf",
    # "https://mediatum.ub.tum.de/doc/1335646/1335646.pdf",
    # "https://mediatum.ub.tum.de/doc/1326486/1326486.pdf",
    # "https://mediatum.ub.tum.de/doc/1306410/1306410.pdf",
    # "https://mediatum.ub.tum.de/doc/1444929/1444929.pdf",
]
    
with TextExtractor("output/processed_documents.json") as extractor:
    # Process local files
    extractor.process_documents(pdf_files)
        
    # Process URLs
    extractor.process_documents(pdf_urls, url_list=True)

Processing documents: 100%|██████████| 1/1 [00:09<00:00,  9.77s/it]
Processing documents: 0it [00:00, ?it/s]


## 2. Prepare MCLM Training Data

In [8]:
# Specify the model and load the tokenizer
# Llama 3.2 3B with approx. 3 billion parameters
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [9]:
# Preprocess the corpus for Causal Language Modeling (CLM)
json_file_paths = ["output/processed_documents.json"]
preprocessor = CLMPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

Token indices sequence length is longer than the specified maximum sequence length for this model (152682 > 131072). Running this sequence through the model will result in indexing errors


In [10]:
# Split into training and test set
train_test_set = dataset.train_test_split(test_size=0.1)
print(f"Created {len(train_test_set['train'])} training examples and {len(train_test_set['test'])} test examples")

# Preview a training example
example = train_test_set["train"][0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids'][:256]))

Created 34 training examples and 4 test examples

Example input:
 Nußelt numbers and friction coefficient. 5.02.03 Data Acquisition To evaluate the dynamic response of the heat transfer rate and skin friction (con- sidered as the “responses” or “outputs”) to free-stream velocity fluctuations (the “signal” or “input”), CFD data was extracted at each time step for subsequent post- processing. As the flow field is considered incompressible, a perturbation in overall mass flow rate imposed at the inlet boundary propagates without delay throughout the domain. Hence, a time lag due to finite propagation speed did not have to be taken into account. A cross-sectional area average of axial velocity u(t) is acquired at a monitor plane 9 d upstream of the cylinder, where the fluid is not influenced noticeably by the presence of the cylinder (cf. Figure 5.02). In order to get a dimensionless value, the velocity is scaled with the cylinder diameter and the kinematic viscosity to form an (unsteady) 

## 3. Load and Prepare Model

In [11]:
# Load the base model
# Q4_K_M quantization of the base model is achieved through BitsAndBytes. It requires CUDA!
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors
    )

base_model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.float16, device_map="auto",
    quantization_config=quantization_config,
    )

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [12]:
# Configure the (Q)LoRA adaptor to use a rank of r=4
model = prepare_model_for_kbit_training(base_model)
config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

## 4. Train the Model

Now we'll fine-tune the model on our domain-specific data.

In [13]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=3, # Creates a virtual batch size of 3
    learning_rate=3e-4,
    fp16=True, # numerical precision of adapter is float16
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit", # Memory efficient optimizer
    log_level="info",
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using auto half precision backend
***** Running training *****
  Num examples = 34
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 3
  Gradient Accumulation steps = 3
  Total optimization steps = 33
  Number of trainable parameters = 6,078,464
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for mo

Epoch,Training Loss,Validation Loss
1,5.843700,5.268306
2,12.183300,4.243965



***** Running Evaluation *****
  Num examples = 4
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-12
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope

TrainOutput(global_step=33, training_loss=17.03787579680934, metrics={'train_runtime': 1770.6516, 'train_samples_per_second': 0.058, 'train_steps_per_second': 0.019, 'total_flos': 6595177422520320.0, 'train_loss': 17.03787579680934, 'epoch': 2.7941176470588234})

In [14]:
# Save the LoRA adapter weights:
lora_save_path = "lora_weights"  # Directory to save the LoRA weights
model.save_pretrained(lora_save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.0",
  "u

## 5. Apply the learned weights to the instruct model

In [15]:
# Load the instruct model
# This model is not quantized!
model_name = "meta-llama/Llama-3.2-3B-Instruct"
instruct_tokenizer = AutoTokenizer.from_pretrained(model_name)
instruct_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config,
)

# Load the LoRA configuration and weights
lora_weights_path = "lora_weights"
peft_config = PeftConfig.from_pretrained(lora_weights_path)

# Apply LoRA adapter to the instruct model
lora_instruct_model = PeftModel.from_pretrained(instruct_model, lora_weights_path)  

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/tokenizer.json
loading file tokenizer.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/tokenizer_config.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Instantiating LlamaForCausalLM model under default dtype torch.float16.
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Llama-3.2-3B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}



## 6. Test the Model

In [16]:
def test_qanda(model, tokenizer):
    # Example queries about heat transfer and fluid dynamics
    queries = [
        "Explain the CFD/SI method.",
        "What is a Rijke tube?",
        "How is a finite impulse response computed?",
        "When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?",
        "For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude peaks in the frequency response at Strouhal numbers between 0-1, and how does this behavior change with Reynolds number?",
    ]
    
    # Generate responses
    for query in queries:
        print(f"\nQuery: {query}")
        response = generate_response(
            model,
            tokenizer,
            query,
            max_new_tokens=512,
            temperature=0.7
        )
        print(f"Response: {response}")
        print("-" * 80)

### 6.1 Answers to the test questions by the instruct model w/o LoRA

In [17]:
test_qanda(instruct_model, instruct_tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: CFD/SI is a method for predicting the behavior of fluid flow in  dimensional spaces. This method is a technique used in of fluid dynamics to, in real, to, and simulations of fluid flow. In this method, the fluid is modeled as a of incompressible fluid, the velocity is represented as velocity, and is modeled as a flow. the of the is defined using velocity, the is defined using, and the is defined. The of is used to the of the flow, the of is used to the of the flow, and the is used to the of the flow. The of is also used to the of the flow, the of is used the of the, and is used the of the..

. method is a and of that the of the is. It is a that the of is. is used to the of, and is used the of. The is also used the of the. The is a that the of is, and is used the of. The is a that the of is, and is used the. The is a that the of is, and is used the.

. method is a that the of is. It is a that the of is. is used the of, and is used the of. The is a that the of is, and is used t

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: (, Rijk, or, )?

A Rijke tube is a type of tube used to study the behavior of electromagnetic waves. It is a tube with two concentric and. The ends are of radius. The tube is made of., which is a material with a high, and low. The is the distance between the inner and outer ends of the. The tube is to study the properties of electromagnetic waves, such as the and of a. The is the distance between the inner and outer ends of the.

The Rijke tube was first by in. It is a of to the. The is a of the properties of electromagnetic waves. The is used to study the behavior of,, and. The is used to the of electromagnetic waves. is a tube of with two concent and. are at and. are at. is the distance between and.. is the distance between.. is the distance between. is the distance between..

## Step 1: Define the Rijke tube
A Rijke tube is a type of tube used to study the behavior of electromagnetic waves.

## Step 2: Describe the components of the Rijke tube
The tube has two concentric c

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: The finite impulse response (FIR) is a of a system by evaluating its response to a sequence of impulses. is a impulse, a a is a impulse. is a, a a a, a. The is the impulse response the a system. to a impulse the system, a a impulse. is a system. response is the of a impulse. to the impulse the system. a. the is. The impulse response a. a system. to a impulse. is. a. a. is. a. a. a a.. response. is. The impulse the. to. the. a. a.. a a.. a. a. a. is.. a. a. a a a a. a. a a a a a. a a a a. a a. a a a. a. a. a. a. a. a a a a a a a. a a a. a. a a. a. a. a. a. a a. a a. a a. a a a a. a. a. a. a a. a. a a a. a a a a. a. a. a. a. a a. a a. a a. a. a. a. a. a a a a a a a a. a a a. a a. a. a. a. a. a a. a a. a a a a. a a a. a. a. a a. a. a a. a a a a. a a a. a a. a. a. a a a a. a a a. a. a. a. a a a. a a a a. a. a. a. a. a. a a a a a a a a a. a a a. a. a. a. a. a a. a a a a. a a a a. a. a. a. a. a a a a. a a a a a. a. a. a. a a a. a a a a. a. a a a. a a. a a a a. a a a. a. a a a. a. a

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: ,  (.,)
## Step, -. - a-.-.- -. a. -.- -. a.- -. a. a- - a- - a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a 

### 6.2 Answers to the test questions by the adapted model

In [18]:
test_qanda(lora_instruct_model, instruct_tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: This method is used to determine the optimal value of a function to maximize or minimize., the CFD, and SI are the inputs, and, and are the variables. is the constraint.., the is a function that maps the variables to a value. is the, a that maps the variables to a. is the of the function. is a. that is., the is a, that is. is a that is., the is the, that is. is the that is., the is a that is. is a that is,. is the that is. is the that is. and the is the that is the, and the are the that are., the is the that, that is. is a that., that is the that, that is. is a that., that is the that, that is. is a that., that is the that, that is. is a that., that is the that, that is. is a that., that is the that, that is the. and the are the that. is a. is. is. is a. is. is. is a. is a. is. is a. is a. is a. is a.. is a. is a. is. is. is. is. is a.. is. is. is. is. a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: a Rijke tube is a a hollow tube that is made of a non- conductive material that is used in a gas flow meter. The tube is connected to a valve and a meter. The flow is then detected by a series of. sensors that detect the flow of gas through the tube. the sensors detect a change in the flow of gas by measuring the change in pressure of the gas in the tube. the pressure is measured by a pressure transducer. the transducer is connected to a meter. The is. a Rijke tube flow meter is commonly used for measuring gas flow in oil and gas applications. A Rijke tube is a type of gas flow meter that uses a tube, a valve, a meter and. sensors to measure the flow of gas. The flow is detected by a series of sensors that detect the flow of gas by measuring the change in pressure the tube. the pressure is measured by a. transducer and the meter is connected. the meter is connected to a. controller. the meter is connected a controller. This meter is commonly used in oil and gas applications. 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: A finite impulse response (FIR) is a type of filter that, when the input is a signal, produces an output signal. The of of FIR is that the impulse response is finite, i.., the impulse response is a function of time that is zero after a certain time. The impulse response is the response of the filter to a impulse input, which is a signal with a single point and is zero everywhere else. to compute FIR, the filter coefficients are set, and the impulse response is computed the the input is a impulse input the filter is applied the impulse is a function of time, and is zero after a certain time the impulse is applied the response is computed the output is signal, and is a function of time.
. FIR are often used in audio processing, signal processing, and image processing.. FIR have the, that they are stable,, they are,,,, and,,. FIR have, that they are,,,,, and,. FIR have, that they,, and,.
 FIR are, that they are,. FIR, are, that they,. FIR, are that they,. FIR, are that they,. FI

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: (,,,)
## Step. The criteria used to validate the models and the minimum performance threshold considered acceptable depend on the specific application and the system being analyzed. However, a common approach is to use a set of validation criteria that include metrics such as accuracy, robustness, and efficiency.

## Step.. The validation criteria may include metrics such as mean absolute percentage (MA), mean (), and root mean (). The minimum performance threshold may be set based on the specific requirements of the application such as, temperature range,, and.

## Step.. a typical validation process may involve comparing the performance of the identified models to that of a, which is an model of the system. the validation process may also involve sensitivity, which is the impact of parameter on the model, and, which is the difference between model output and actual output.

##. The criteria used to validate the models and the minimum performance threshold considered accepta